# Домашнее задание по теме "Постановка задачи" (часть 4)

Цель этого задания - привлечь внимание к одному из важных шагов постановки задачи машинного обучения: выбору метрики. Для этого предлагается изучить, как модель может меняться в зависимости от оптимизируемой метрики качества. Лучше всего это заметно на примере простых моделей, поэтому мы начнем с решения задачи регрессии с помощью константного прогноза. 

Затем мы рассмотрим задачу классификации с несбалансированными классами и изучим значения различных метрик для одной и той же прогнозной модели, а после этого - решения, которые получаются при оптимизации различных метрик качества по гиперпараметрам прогнозной модели.

Допишите код там, где это требуется (отмечено комментариями # TODO) и ответьте на вопросы, приведенные в этом блокноте

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Метрики регрессии

Создадим синтетические данные с выбросами

In [ ]:
gen = np.random.RandomState(42)
data = 1 + np.concatenate((
    np.abs(gen.normal(loc=0., scale=100, size=10000)), # основные данные
    np.abs(30000 * gen.standard_cauchy(size=10)) # небольшая часть выбросов
))

Напишите реализацию метрик регрессии (попробуйте не использовать sklearn)

In [ ]:
def mae(answers, predictions):
    # TODO допишите реализацию
    

def rmse(answers, predictions):
    # TODO допишите реализацию
    

def mape(answers, predictions):
    # TODO допишите реализацию
    

def smape(answers, predictions):
    # TODO допишите реализацию
    

Визуализируем зависимость качества от константы, которой мы будем предсказывать целевые переменные

In [ ]:
def plot(x_coords, y_coords):
    plt.figure(figsize=(10, 6))
    plt.ylim(np.min(y_coords), np.max(y_coords))
    plt.xlim(np.min(x_coords), np.max(x_coords))
    plt.plot(x_coords, y_coords)
    plt.grid()
    plt.show()

In [ ]:
x_coords = list(range(0, 250, 5)) # диапазон значений

In [ ]:
plot(x_coords, [mae(data, x) for x in x_coords])

In [ ]:
plot(x_coords, [rmse(data, x) for x in x_coords])

In [ ]:
plot(x_coords, [mape(data, x) for x in x_coords])

In [ ]:
plot(x_coords, [smape(data, x) for x in x_coords])

Объясните наблюдаемые эффекты для каждого графика:
* Почему кривая имеет такую форму?
* Где её точный оптимум (возможно, придётся написать дополительный код)?
* Какие выводы можно сделать с точки зрения использования метрики для оценки алгоритма?

In [ ]:
# TODO: получите и выведите на экран оптимальные значения метрик


In [ ]:
# TODO: напишите выводы

## Метрики классификации при дисбалансе классов

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

Воспользуемся sklearn реализацией метрик

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

Возьмём стандартный датасет для классификации из sklearn

In [ ]:
X_data, y_data = load_breast_cancer(return_X_y=True)

Разделим на train и test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.5, random_state=42)

Импортируем алгоритм для обучения

In [ ]:
from xgboost import XGBClassifier

Строим модель

In [ ]:
model = XGBClassifier(n_estimators=1000, random_state=42).fit(X_train, y_train)

In [ ]:
accuracy_scores = []
recall_scores = []
precision_scores = []
f1_scores = []
roc_auc_scores = []

points = list(range(1, 101))

for k in points:
    # TODO постройте список индексов, такой, чтобы каждый 
    # индекс объекта с ответом 0 попал в тест один раз, 
    # а каждый индекс объекта с ответом 1 попал в тест k раз
    # (создаем дисбалланс классов в тесте)
    indices = 
    
    X = X_test[indices, :]
    y = y_test[indices]
    
    y_pred = model.predict(X)
    accuracy_scores.append(accuracy_score(y, y_pred))
    recall_scores.append(recall_score(y, y_pred))
    precision_scores.append(precision_score(y, y_pred))
    f1_scores.append(f1_score(y, y_pred))
    roc_auc_scores.append(roc_auc_score(y, y_pred))
        

Отобразим кривые в зависимости от $k$

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(points, accuracy_scores)
plt.plot(points, recall_scores)
plt.plot(points, precision_scores)
plt.plot(points, f1_scores)
plt.plot(points, roc_auc_scores)
plt.grid()
plt.legend([
    'accuracy_scores',
    'recall_scores',
    'precision_scores',
    'f1_scores',
    'roc_auc_scores'
], loc='best')
plt.show()

* Почему кривые такие? Почему некоторые не меняются, а некоторые меняются
* Почему у кривых такие асимптотические значения? Почему у некоторых они совпадают?
* Какие выводы можно сделать с точки зрения использования метрики для оценки алгоритма?

In [ ]:
# TODO: напишите выводы

### Оптимальные гиперпараметры при обучении

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [ ]:
def estimate_algo(algo):
    print(algo.best_params_)
    print('Train:')
    y_pred = algo.predict(X_train)
    print('\taccuracy_score', accuracy_score(y_train, y_pred))
    print('\trecall_score', recall_score(y_train, y_pred))
    print('\tprecision_score', precision_score(y_train, y_pred))
    print('\tf1_score', f1_score(y_train, y_pred))
    print('\troc_auc_score', roc_auc_score(y_train, y_pred))
    print('Test:')
    y_pred = algo.predict(X_test)
    print('\taccuracy_score', accuracy_score(y_test, y_pred))
    print('\trecall_score', recall_score(y_test, y_pred))
    print('\tprecision_score', precision_score(y_test, y_pred))
    print('\tf1_score', f1_score(y_test, y_pred))
    print('\troc_auc_score', roc_auc_score(y_test, y_pred))
    

Ниже будут перебираться гиперпараметры и находиться оптимальный алгоритм. Почему получаются именно такие гиперпараметры?

In [ ]:
print('accuracy optimal')
estimate_algo(GridSearchCV(
    XGBClassifier(random_state=42), 
    param_grid=dict(
        n_estimators=[5, 10, 20, 25],
        max_depth=[1, 2, 3, 4, 5, 6, 7],
        learning_rate=[0.1, 0.2, 0.05, 0.3, 0.4]
    ),
    cv=2, scoring=make_scorer(accuracy_score), iid=False
).fit(X_train, y_train))

In [ ]:
print('recall optimal')
estimate_algo(GridSearchCV(
    XGBClassifier(random_state=42), 
    param_grid=dict(
        n_estimators=[5, 10, 20, 25],
        max_depth=[1, 2, 3, 4, 5, 6, 7],
        learning_rate=[0.1, 0.2, 0.05, 0.3, 0.4]
    ),
    cv=2, scoring=make_scorer(recall_score), iid=False
).fit(X_train, y_train))

In [ ]:
print('precision optimal')
estimate_algo(GridSearchCV(
    XGBClassifier(random_state=42), 
    param_grid=dict(
        n_estimators=[5, 10, 20, 25],
        max_depth=[1, 2, 3, 4, 5, 6, 7],
        learning_rate=[0.1, 0.2, 0.05, 0.3, 0.4]
    ),
    cv=2, scoring=make_scorer(precision_score), iid=False
).fit(X_train, y_train))

In [ ]:
print('roc_auc optimal')
estimate_algo(GridSearchCV(
    XGBClassifier(random_state=42), 
    param_grid=dict(
        n_estimators=[5, 10, 20, 25],
        max_depth=[1, 2, 3, 4, 5, 6, 7],
        learning_rate=[0.1, 0.2, 0.05, 0.3, 0.4]
    ),
    cv=2, scoring=make_scorer(roc_auc_score), iid=False
).fit(X_train, y_train))

In [ ]:
# TODO: напишите выводы